In [1]:
import joblib
import torch
import numpy
from BertModel import QueryClassifier
from utils import *
import cvxpy as cp
import time


/home/vk352/anaconda3/envs/DSI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentence_embeddings = joblib.load('/home/vk352/dsi/outputs/dpr5_finetune_0.001_filtered_fixed/nq320k_gen_passage_embeddings.pkl')


In [3]:
class_num = 100001
model = QueryClassifier(class_num)
initialize_model = "/home/vk352/dsi/outputs/dpr5_finetune_0.001_filtered_fixed/projection_nq320k_epoch15"
load_saved_weights(model, initialize_model, strict_set=False)
classifier_layer = model.classifier.weight.data

Some weights of the model checkpoint at bert-base-uncased were not used when initializing HFBertEncoder: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing HFBertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HFBertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
embeddings = sentence_embeddings[:1000010][::10]
embeddings_new = sentence_embeddings[1000010:][::10]

In [5]:
sentence_embeddings.shape, embeddings.shape, embeddings_new.shape, classifier_layer.shape

(torch.Size([1097150, 768]),
 torch.Size([100001, 768]),
 torch.Size([9714, 768]),
 torch.Size([100001, 768]))

In [6]:
# sanity check to ensure old docs are classified correctly
q = sentence_embeddings[1000010]
torch.max(torch.matmul(classifier_layer, sentence_embeddings[1000010]))

tensor(70.3474)

In [25]:
# cvxpy optimization
q = sentence_embeddings[1000010]

x = cp.Variable(768)
objective = cp.Minimize(cp.norm2(x))
constraints = [(q.unsqueeze(dim=0) @ x) >= 74,  (embeddings-classifier_layer) @ x + 0.5 <= 0]
prob = cp.Problem(objective, constraints)

In [26]:
start = time.time()
result = prob.solve(solver=cp.MOSEK)
print(time.time() - start)

221.43086218833923


In [14]:
x.value

array([6.02428738e-15, 1.77742540e-15, 7.94738612e-15, 6.58186626e-15,
       8.37354743e-15, 1.21204980e-14, 1.73382705e-14, 1.95942463e-15,
       6.13761646e-15, 7.94192033e-15, 7.72882056e-15, 7.77652042e-15,
       4.68132347e-16, 8.50078837e+00, 1.94422594e-14, 3.81761953e-15,
       1.90866167e-14, 6.01731626e-15, 1.16539200e-14, 9.07017863e-15,
       5.80164370e-15, 2.58721338e-14, 5.85521177e-14, 5.20099685e-15,
       1.89169587e-14, 1.74695000e-14, 1.79556065e-14, 3.24465448e-14,
       1.84946766e-13, 1.04676287e+00, 2.30562089e-14, 9.67793550e-15,
       5.11285370e-15, 4.46041437e-15, 7.38650739e-15, 1.04169708e-13,
       1.50460920e-14, 5.01637686e-15, 4.60429996e-15, 1.47838122e-14,
       6.92069467e-15, 9.08186100e-14, 3.82472406e-15, 1.64733364e-14,
       2.18308874e-14, 1.87915781e-14, 0.00000000e+00, 3.12815696e-14,
       1.06133743e-14, 1.40079229e-14, 1.18058538e-14, 1.52898770e-14,
       8.12611648e-15, 6.86035001e-14, 6.51850134e-15, 1.67062564e-14,
      

In [3]:
cp.installed_solvers()

['ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']

In [7]:
# see how many documents we can add

In [ ]:
for i in range(len(embeddings_new)):
    q = sentence_embeddings[i]
    max_val = torch.max(torch.matmul(classifier_layer, q))
    
    x = cp.Variable(768)
    objective = cp.Minimize(cp.norm2(x))
    constraints = [(q.unsqueeze(dim=0) @ x) >= (max_val+0.5),  (embeddings-classifier_layer) @ x + 0.5 <= 0]
    prob = cp.Problem(objective, constraints)
    
    start = time.time()
    result = prob.solve(solver=cp.MOSEK)
    print(f'Done {i} in {time.time() - start} seconds')
    
    # add to classifier_layer and embeddings
    classifier_layer = torch.cat((classifier_layer, torch.tensor(x.value).unsqueeze(dim=0))).float()
    embeddings = torch.cat((embeddings, q.unsqueeze(dim=0)))

Done 0 in 197.26723670959473 seconds
Done 1 in 196.19284057617188 seconds
Done 2 in 198.5630145072937 seconds
Done 3 in 196.68159103393555 seconds
Done 4 in 195.32428908348083 seconds
Done 5 in 194.87955904006958 seconds
Done 6 in 197.15087628364563 seconds
Done 7 in 198.03854084014893 seconds
Done 8 in 198.872150182724 seconds
Done 9 in 196.14507460594177 seconds
Done 10 in 196.06368207931519 seconds
Done 11 in 221.22731828689575 seconds
Done 12 in 217.04163002967834 seconds
Done 13 in 220.75112509727478 seconds
Done 14 in 234.05558514595032 seconds
Done 15 in 225.97347903251648 seconds
Done 16 in 219.26720809936523 seconds
Done 17 in 196.33256077766418 seconds
Done 18 in 205.22683811187744 seconds
Done 19 in 198.09482431411743 seconds
Done 20 in 200.39648127555847 seconds
Done 21 in 205.50771117210388 seconds
Done 22 in 210.30535173416138 seconds
Done 23 in 212.40150451660156 seconds
Done 24 in 191.22746109962463 seconds
Done 25 in 192.8119192123413 seconds
Done 26 in 195.03355312347

In [39]:
torch.tensor(x.value)

tensor([-1.0447e-01, -4.0969e-01, -9.5017e-02, -1.0921e-01,  8.1158e-03,
         6.1220e-02,  1.9154e-01, -1.1731e-01,  3.7474e-02, -3.7682e-02,
        -2.3466e-01, -5.5951e-02, -2.1378e-01,  3.1935e-01,  2.3516e-01,
        -3.0110e-02,  1.2662e-01,  7.7644e-02,  1.7344e-01, -9.7156e-02,
        -8.4197e-02,  5.4189e-02,  1.3672e-01, -1.5080e-01,  1.3905e-02,
         9.6931e-02,  1.4062e-01,  1.3511e-01,  6.5622e-02,  1.1421e-01,
         1.2426e-01,  1.1707e-02, -8.7997e-03, -1.3500e-01, -6.9370e-02,
        -3.6810e-02,  7.5156e-03, -2.6950e-01, -6.7042e-02,  6.9721e-02,
        -1.8917e-01,  2.2150e-01, -1.3268e-01,  1.6955e-01,  4.8951e-02,
         1.5185e-01, -6.0079e-01,  1.3380e-01, -1.0681e-01, -9.9943e-02,
        -8.4951e-02,  1.9041e-01,  1.2308e-01,  1.5919e-01,  3.0746e-02,
         1.9739e-01, -1.6027e-01,  3.4278e-02, -2.1023e-01, -6.1769e-02,
         2.6638e-01,  1.3060e-01,  1.3632e-01,  2.3209e-01,  2.4003e-01,
         3.2724e-03,  1.0809e-01,  3.1118e-01, -2.2